# Tutorial: CNN with Pytorch using Opp dataset

In [1]:
import sys
sys.path.append('../')

In [2]:
# general
import numpy as np
import pandas as pd
from progressbar import ProgressBar

# dataset
from actreco.datasets.opportunityc import Opportunity
from actreco.utils.generator import batch_generator

# modeling
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


In [3]:
# dataset

In [13]:
train_set = Opportunity(userID='S2,S3,S4')
x_train_list = train_set.get('X')
y_train_list = train_set.get('y')

In [14]:
test_set = Opportunity(userID='S1')
x_test_list = test_set.get('X')
y_test_list = test_set.get('y')

In [15]:
class VanilaCNN(nn.Module):
    def __init__(self, l_sample, nb_modal, nb_in_filter, nb_filter_list, kw, nb_unit, nb_out):
        super(VanilaCNN, self).__init__()
        self.l_sample = l_sample
        self.nb_modal = nb_modal
        self.nb_conv = len(nb_filter_list)
        
        if isinstance(kw, int):
            kw = [kw] * self.nb_conv
        assert len(nb_filter_list) == len(kw)
        nb_filter_list = [nb_in_filter] + nb_filter_list
        
        self.kw = kw
        self.nb_filter_list = nb_filter_list

        convs = []
        for i in range(self.nb_conv):
            convs.append(nn.Conv2d(nb_filter_list[i], nb_filter_list[i+1], (1, kw[i])))
        self.convs = nn.ModuleList(convs)
        self.fc1 = nn.Linear(self.conv_output_shape(), nb_unit)
        self.fc2 = nn.Linear(nb_unit, nb_out)
    
    def forward(self, x):
        for conv in self.convs:
            x = F.max_pool2d(F.relu(conv(x)), (1, 2))
            dropout = nn.Dropout2d(0.5)
            # x = dropout(x)
        x = x.view(-1, self.conv_output_shape())
        x = F.dropout(self.fc1(x), 0.5)
        x = F.relu(x)
        x = self.fc2(x)
        return x
    
    def conv_output_shape(self):
        length = self.l_sample
        for i in range(self.nb_conv):
            length = int((length - self.kw[i] + 1) /2)
        return length * self.nb_filter_list[-1] * self.nb_modal

In [28]:
nb_modal_dim = 2
nb_modal = train_set.nb_modal
nb_out = train_set.nb_class

kw = 3
nb_filter_list = [50, 40, 20]
nb_in_filter = 1
drop_rate = 0.5
nb_unit = 400

batch_size = 128
l_sample = 30
interval = int(l_sample) * 0.5

nb_iter = 2000
report_each = 10

model = VanilaCNN(l_sample=l_sample, nb_modal=nb_modal, nb_in_filter=1, nb_filter_list=nb_filter_list, kw=kw, nb_unit=nb_unit, nb_out=nb_out).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

In [33]:
model.train()

p = ProgressBar(max_value=nb_iter)
gen = batch_generator(x_train_list, y_train_list, batch_size, l_sample, train=True, nb_iter=nb_iter, categorical=False)
for batch_idx, (X, y) in enumerate(gen):
    batch_idx += 1
    tX = Variable(torch.from_numpy(X.astype('float32')), requires_grad=False).cuda()
    ty = Variable(torch.LongTensor(y), requires_grad=False).cuda()
    
    optimizer.zero_grad()
    output = model(tX)
    
    loss = criterion(output, ty)
    loss.backward()
    optimizer.step()
    if batch_idx % report_each == 0:
        print("{} samples, {} batch \t Loss: {:.6f}".format(batch_idx * batch_size, batch_idx, loss.data[0]))
    p.update(batch_idx)
    

  0% (14 of 2000) |                       | Elapsed Time: 0:00:00 ETA:  0:01:24

1280 samples, 10 batch 	 Loss: 0.283480


  1% (23 of 2000) |                       | Elapsed Time: 0:00:00 ETA:  0:01:23

2560 samples, 20 batch 	 Loss: 0.301822


  1% (34 of 2000) |                       | Elapsed Time: 0:00:01 ETA:  0:01:22

...

253440 samples, 1980 batch 	 Loss: 0.128508


 99% (1995 of 2000) |#################### | Elapsed Time: 0:01:23 ETA:  0:00:00

254720 samples, 1990 batch 	 Loss: 0.126635


100% (2000 of 2000) |#####################| Elapsed Time: 0:01:23 ETA:  0:00:00

256000 samples, 2000 batch 	 Loss: 0.193492


In [30]:
from sklearn import metrics

def test(model, x_list, y_list, l_sample, nb_iter=1, batch_size=2048):
    model.eval()
    test_gen = batch_generator(x_list, y_list, batch_size, l_sample, train=False, nb_iter=nb_iter, categorical=False, seed=0)
    y_list = []
    py_list = []
    for X, y in test_gen:
        tX = Variable(torch.from_numpy(X.astype('float32')), requires_grad=False).cuda()
        py = model(tX).max(1)[1].data.cpu().numpy().reshape(-1)
        y_list.append(y)
        py_list.append(py)
    
    y = np.concatenate(y_list)
    py = np.concatenate(py_list)
    print(metrics.accuracy_score(y, py))
    print(metrics.f1_score(y, py, average='macro'))
    print(metrics.precision_score(y, py, average=None))
    print(metrics.recall_score(y, py, average=None))

In [32]:
test(model, x_test_list, y_test_list, l_sample, nb_iter=10, batch_size=batch_size)

0.77109375
0.561022799081
[ 0.91304348  0.83333333  0.82352941  0.76666667  0.42253521  0.22857143
  0.28571429  0.36666667  0.5         1.          0.35294118  0.33333333
  1.          0.66666667  0.83333333  0.67741935  0.80952381  0.85028571]
[ 0.7         0.74074074  0.60869565  0.88461538  0.90909091  0.42105263
  0.15384615  0.91666667  0.46153846  0.36842105  0.35294118  0.22222222
  0.23529412  0.45454545  0.83333333  0.32307692  0.60714286  0.90953545]


In [11]:
# keras model for comparison
# which is much slower than pytorch

In [12]:
import keras
if keras.__version__ >= '2.0.0':

    from keras.models import Sequential
    from keras.layers import Dense, Convolution2D, Dropout, Activation, Flatten
    from keras.layers import MaxPooling2D

    from keras.optimizers import SGD

    model = Sequential()
    model.add(Convolution2D(50, kernel_size=(1, 3), input_shape=[1, nb_modal, l_sample]))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(1, 2)))
    model.add(Convolution2D(40, (1, 3)))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(1, 2)))
    model.add(Convolution2D(20, (1, 3)))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(1, 2)))
    model.add(Flatten())
    model.add(Dense(nb_unit))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_out))
    model.add(Activation('softmax'))
    model.summary()

    optimizer = SGD(lr=0.01, momentum=0.5)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy')
    
    p = ProgressBar(max_value=report_each)
    gen = batch_generator(x_train_list, y_train_list, batch_size, l_sample, train=True, nb_iter=nb_iter, categorical=train_set.nb_class)
    for batch_idx, (X, y) in enumerate(gen):
        batch_idx += 1
        loss = model.train_on_batch(X, y)
        if batch_idx % report_each == 0:
            print("Seen {} samples, {} batch \t Loss: {:.6f}".format(batch_idx * batch_size, batch_idx, float(loss)))
            p.update(report_each)
        else:
            p.update((batch_idx % report_each))
else:
    print("TBA")

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



TBA


Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN Mixed dnn version. The header is from one version, but we link with a different version (4007, 6021))
